In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import pyspark
from pyspark.sql.types import NumericType, IntegerType
import pandas as pd

import pyspark.sql
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import VectorAssembler

In [42]:
unique_genres = set()
for genres in movies['genres'].str.split('|'):
    unique_genres.update(genres)
unique_genres.remove('(no genres listed)')

def movie_similarity(movie1_id, movie2_id, movies):
    if isinstance(movie1_id, str):
        movies[movies['']]
    return None

In [43]:
unique_genres

{'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [3]:
movies = pd.read_csv('../data/csv/movies.csv')

In [4]:
ratings = pd.read_csv('../data/csv/ratings.csv')

In [5]:
tags = pd.read_csv('../data/csv/tags.csv')

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
tags = tags.drop('timestamp', axis = 1)
tags

,userId,movieId,tag
0,2,60756,funny
1,2,60756,Highly quotable
2,2,60756,will ferrell
3,2,89774,Boxing story
4,2,89774,MMA
...,...,...,...
3678,606,7382,for katie
3679,606,7936,austere
3680,610,3265,gun fu
3681,610,3265,heroic bloodshed


In [136]:
def tag_encoder(tags):
    user_temp = []
    movie_temp = []
    tags_temp = []
    unique_tags = set()
    for user, i in tags.groupby('userId'):
        for movie, j in i.groupby('movieId'):
            user_temp.append(user)
            movie_temp.append(movie)
            tags_temp.append(list(j['tag']))
            unique_tags.update(list(j['tag']))

    tag_frame = pd.DataFrame(columns = unique_tags)
    encoded_tags = pd.DataFrame(zip(user_temp, movie_temp), columns = ['userId', 'movieId'])
    encoded_tags = pd.merge(encoded_tags, tag_frame, how = 'left', left_index = True, right_index = True).fillna(0)

    for index in range(len(tags_temp)):
        for tag in tags_temp[index]:
            encoded_tags.at[index, tag] = 1
            
    return encoded_tags

def genre_encoder(movies):
    genres = []
    unique_genres = set()
    for _, row in movies.iterrows():
        current_genres = row['genres'].split('|')
        genres.append(current_genres)
        unique_genres.update(current_genres)
        
    genre_frame = pd.DataFrame(columns = unique_genres)
    encoded_genres = pd.merge(movies[['movieId', 'title']], genre_frame, how = 'left', left_index = True, right_index = True).fillna(0)

    for index in range(len(genres)):
        for genre in genres[index]:
            encoded_genres.at[index, genre] = 1
            
    return encoded_genres

,movieId,title,Action,War,Fantasy,Horror,Musical,(no genres listed),Western,Romance,...,Mystery,Sci-Fi,Crime,Drama,IMAX,Comedy,Film-Noir,Children,Adventure,Thriller
0,1,Toy Story (1995),0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
1,2,Jumanji (1995),0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9738,193583,No Game No Life: Zero (2017),0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
encoded_tags = tag_encoder(tags)
encoded_tags.loc[0]['funny']

1

In [11]:
joined = movies.set_index('movieId').join(ratings.set_index('movieId'))
joined = joined[~joined['userId'].isna()] 
joined

,title,genres,userId,rating,timestamp
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
193585,Flint (2017),Drama,184.0,3.5,1.537110e+09


In [91]:
merged = pd.merge(ratings, encoded_tags, how = 'left', left_on = ['userId','movieId'], right_on = ['userId','movieId'])
merged = pd.merge(merged, movies, how = 'left', on = 'movieId').fillna(0)
merged

,userId,movieId,rating,timestamp,powerful ending,Harper Lee,abstract,alternate universe,POW,invisibility,...,character study,Beatles,alter ego,show business,Louisa May Alcott,imdb top 250,awkward romance,stand-up comedy,title,genres
0,1,1,4.0,964982703,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Logan (2017),Action|Sci-Fi


In [13]:
trimmed = joined.reset_index()[['movieId', 'userId', 'rating']]
trimmed

,movieId,userId,rating
0,1,1.0,4.0
1,1,5.0,4.0
2,1,7.0,4.5
3,1,15.0,2.5
4,1,17.0,4.5
...,...,...,...
100831,193581,184.0,4.0
100832,193583,184.0,3.5
100833,193585,184.0,3.5
100834,193587,184.0,3.5


In [10]:
users_set = trimmed['userId'].unique()
users_set.sort()

movie_set = trimmed['movieId'].unique()
movie_set.sort()

user_frame = pd.DataFrame(index = users_set, columns = movie_set)
user_frame = user_frame.fillna(0)

counter = 0
ten_precent = int(trimmed.shape[0] / 10)
for row in trimmed.iterrows():
    counter += 1
    if counter % ten_precent == 0:
        print(counter // ten_precent * 10, '% of the way done')
    row = row[1]
    _userId = row['userId']
    _movieId = row['movieId']
    user_frame.loc[_userId][_movieId] = row['rating']

10 % of the way done
20 % of the way done
30 % of the way done
40 % of the way done
50 % of the way done
60 % of the way done
70 % of the way done
80 % of the way done
90 % of the way done
100 % of the way done


In [11]:
user_num = 5
new_user = user_frame.loc[user_num]
new_frame = user_frame.copy().drop(user_num)

sims = []
for row in new_frame.iterrows():
    userId = row[0]
    row = row[1]
    sims.append((userId, cosine_similarity([row], [new_user])[0][0]))
    
sims.sort(key = lambda x: x[1], reverse = True)



def get_top_5(user_frame, user_info, userId_other, movies):
    user_info_other = user_frame.iloc[userId_other]

    m = (user_info_other * user_info)
    top_five = m[m > 0].sort_values(ascending = False)[0: 5]

    for movie_id in top_five.index:
        top_five.loc[movie_id] = movies.set_index('movieId').loc[movie_id]['title']
    return top_five

top_five = []
index = 0
while len(top_five) < 5 and index < len(sims):
    closest_user = int(sims[index][0])
    new_movies = list(get_top_5(new_frame, new_user, closest_user, movies))
    for movie in new_movies:
        if movie not in top_five:
            top_five.append(movie)
    index += 1
    
top_five = top_five[:5]

In [12]:
top_five

['Usual Suspects, The (1995)',
 'Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)"]

In [62]:
#old code
'''
users_set = trimmed['userId'].unique()
users_set.sort()
user_frame = pd.DataFrame(users_set, columns = ['userId'])
user_frame = user_frame.set_index('userId')

movies_set = trimmed['movieId'].unique()
movies_set.sort()
movie_vector_blank = pd.DataFrame([np.zeros(len(movies_set))], columns = movies_set)

user_frame['movie_vect'] = [movie_vector_blank.copy() for i in range(user_frame.shape[0])]

counter = 0
ten_precent = int(trimmed.shape[0] / 10)
for row in trimmed.iterrows():
    counter += 1
    if counter % ten_precent == 0:
        print(counter // ten_precent * 10, '% of the way done')
    row = row[1]
    _userId = row['userId']
    _movieId = row['movieId']
    user_frame.loc[_userId]['movie_vect'][_movieId] = row['rating']
'''

10 % of the way done
20 % of the way done
30 % of the way done
40 % of the way done
50 % of the way done
60 % of the way done
70 % of the way done
80 % of the way done
90 % of the way done
100 % of the way done


In [66]:
#old code
'''
user_num = 1
new_user = user_frame.loc[user_num]['movie_vect']
new_frame = user_frame.copy().drop(user_num)

sims = []
for row in new_frame.iterrows():
    userId = row[0]
    row = row[1]
    sims.append((userId, cosine_similarity(row['movie_vect'], new_user)[0][0]))
    
sims.sort(key = lambda x: x[1], reverse = True)
closest_user = int(sims[0][0])
closest_user = new_frame.iloc[closest_user]['movie_vect']

m = (closest_user * new_user).T
top_five = m[m[0] > 0].sort_values(by = 0, ascending = False)[0: 5]

for movie_id in top_five.index:
    top_five.loc[movie_id] = movies.set_index('movieId').loc[movie_id]['title']
'''


In [14]:
spark = SparkSession(SparkContext())

ratings = spark.createDataFrame(trimmed)
(training, test) = ratings.randomSplit([0.8, 0.2])

als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="reviewerID_index",itemCol="asin_index",ratingCol="overall",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:220)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.Abstract